In [424]:
!pip install opendatasets

In [425]:
import opendatasets as od
import cv2
import numpy as np

od.download(
    "https://www.kaggle.com/competitions/spaceship-titanic/data"
)

Skipping, found downloaded files in "./spaceship-titanic" (use force=True to force download)


In [426]:
import pandas as pd
train=pd.read_csv("/content/spaceship-titanic/train.csv")
test=pd.read_csv("/content/spaceship-titanic/test.csv")


In [427]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [428]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [429]:
# drop verbose features
train=train.drop(['PassengerId','Cabin','CryoSleep','Name'],axis=1)

In [430]:
train.isnull().sum()

HomePlanet      201
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [431]:
from sklearn.preprocessing import OrdinalEncoder

oe=OrdinalEncoder()
train['HomePlanet']=oe.fit_transform(train[['HomePlanet']])
train['Destination']=oe.fit_transform(train[['Destination']])
train['VIP']=oe.fit_transform(train[['VIP']])
train['Transported']=oe.fit_transform(train[['Transported']])

In [432]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   float64
 1   Destination   8511 non-null   float64
 2   Age           8514 non-null   float64
 3   VIP           8490 non-null   float64
 4   RoomService   8512 non-null   float64
 5   FoodCourt     8510 non-null   float64
 6   ShoppingMall  8485 non-null   float64
 7   Spa           8510 non-null   float64
 8   VRDeck        8505 non-null   float64
 9   Transported   8693 non-null   float64
dtypes: float64(10)
memory usage: 679.3 KB


In [433]:
train

,HomePlanet,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0
3,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...
8688,1.0,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0.0
8689,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8690,0.0,2.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1.0
8691,1.0,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0.0


In [434]:
from sklearn.impute import KNNImputer
mssi=KNNImputer(n_neighbors=5, weights="uniform")
train['HomePlanet']=mssi.fit_transform(train[['HomePlanet']])
train['VIP']=mssi.fit_transform(train[['VIP']])
train['Age']=mssi.fit_transform(train[['Age']])
train['RoomService']=mssi.fit_transform(train[['RoomService']])
train['FoodCourt']=mssi.fit_transform(train[['FoodCourt']])
train['ShoppingMall']=mssi.fit_transform(train[['ShoppingMall']])
train['Spa']=mssi.fit_transform(train[['Spa']])
train['VRDeck']=mssi.fit_transform(train[['VRDeck']])
train['Destination']=mssi.fit_transform(train[['Destination']])

In [435]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train.drop(['Transported'],axis=1),train['Transported'],test_size=0.2,random_state=0)

In [436]:
from sklearn.ensemble import AdaBoostClassifier
ac=AdaBoostClassifier(n_estimators=1000,learning_rate=0.4,algorithm='SAMME',random_state=0)
ac.fit(x_train,y_train)
y_predicted=ac.predict(x_test)

In [437]:
from sklearn.metrics import accuracy_score
accuracy_score(y_predicted,y_test)

0.78205865439908

In [438]:
from sklearn.metrics import classification_report
prediction = ac.predict(x_test)
print(classification_report(prediction, y_test))

              precision    recall  f1-score   support

         0.0       0.72      0.82      0.77       758
         1.0       0.84      0.75      0.80       981

    accuracy                           0.78      1739
   macro avg       0.78      0.79      0.78      1739
weighted avg       0.79      0.78      0.78      1739



In [439]:
pid=test['PassengerId']
test=test.drop(['PassengerId','Cabin','CryoSleep','Name'],axis=1)

test['HomePlanet']=oe.fit_transform(test[['HomePlanet']])
test['Destination']=oe.fit_transform(test[['Destination']])
test['VIP']=oe.fit_transform(test[['VIP']])


test['HomePlanet']=mssi.fit_transform(test[['HomePlanet']])
test['VIP']=mssi.fit_transform(test[['VIP']])
test['Age']=mssi.fit_transform(test[['Age']])
test['RoomService']=mssi.fit_transform(test[['RoomService']])
test['FoodCourt']=mssi.fit_transform(test[['FoodCourt']])
test['ShoppingMall']=mssi.fit_transform(test[['ShoppingMall']])
test['Spa']=mssi.fit_transform(test[['Spa']])
test['VRDeck']=mssi.fit_transform(test[['VRDeck']])
test['Destination']=mssi.fit_transform(test[['Destination']])

y=ac.predict(test)

In [440]:
y=y.astype(str)
y[y=='1.0']='True'
y[y=='0.0']='False'
answer=pd.DataFrame({
    'PassengerId':pid,
    'Transported':y
})
answer.to_csv('./answer.csv',sep=',',index=False,header=True)